# Практическая часть

In [5]:
from Bio import Phylo
from Bio.Phylo.Newick import Clade
from io import StringIO
from PrettyPrint import PrettyPrintTree # pip install PrettyPrintTree
import numpy as np

In [15]:
class Tree:
    def __init__(self, value):
        self.val = value
        self.children = []

    def add_child(self, child):
        self.children.append(child)
        return child
    
    def print(self):
        pt = PrettyPrintTree(lambda x: x.children, 
                             lambda x: x.val, 
                             default_orientation=PrettyPrintTree.HORIZONTAL)
        pt(self)

    
def build_tree(root: Clade):
    if root.name is not None:
        return root.name, Tree(root.name)
    letters = set()
    tmp = Tree(None)
    for clade in root.clades:
        let, tree = build_tree(clade)
        tmp.add_child(tree)
        if not letters.intersection(let):
            letters = letters.union(let)
        else:
            letters = letters.intersection(let)
    tmp.val = letters
    return letters, tmp

def choose(root: Tree, parent_val=None):
    if type(root.val) == set:
        if parent_val is None:
            root.val = np.random.choice(list(root.val))
        else:
            if parent_val in root.val:
                root.val = parent_val
            else:
                root.val = np.random.choice(list(root.val))
    for i in range(len(root.children)):
        root.children[i] = choose(root.children[i], root.val)
    return root


In [25]:
treedata = "(((A, A), C), (C, G))"
tree = Phylo.read(StringIO(treedata), "newick")

Находим какие могут быть вершины

In [26]:
tree_obj = build_tree(tree.root)[1]
tree_obj.print()

                             ┌ A 
                     ┌ {'A'} ┤
        ┌ {'A', 'C'} ┤       └ A 
        │            │
- {'C'} ┤            └ C 
        │
        │            ┌ C 
        └ {'C', 'G'} ┤
                     └ G 


Выбираем вершины в соответствии с принципом максимальной парсимонии

In [27]:
choose(tree_obj).print()

            ┌ A 
        ┌ A ┤
    ┌ C ┤   └ A 
    │   │
- C ┤   └ C 
    │
    │   ┌ C 
    └ C ┤
        └ G 


Пример из видео про алгоритм

In [31]:
treedata = "(A, ((A, (G, T)), (C, T)))"

choose(build_tree(Phylo.read(StringIO(treedata), "newick").root)[1]).print()

    ┌ A 
    │
    │       ┌ A 
    │       │
    │   ┌ T ┤   ┌ G 
- A ┤   │   └ T ┤
    └ T ┤       └ T 
        │
        │   ┌ C 
        └ T ┤
            └ T 


# Теоретическая часть

### Укорененные деревья

Мы рассматриваем бинарные деревья

В левого ребенка мы можем поместить $i = 1...n-1$ листьев, а в правого $n - i$ листьев. Тк нам не важен порядок листьев, то всего способов разместить n листьев по двум детям равно:

$$
\frac{1}{2} \sum_{i=1}^{n-1} \binom{n}{i}
$$

i - число листьев в левом ребенке. И 1/2 тк не важен порядок листьев.

Тогда T(n) получим рекурсивно:

$$
T(n) = \frac{1}{2} \sum_{i=1}^{n-1} \binom{n}{i} T(i) T(n-i)
$$

In [38]:
from scipy.special import binom

In [67]:
def calc_T(n):
    if n == 1:
        return 1
    return sum(0.5*binom(n, i)*calc_T(i)*calc_T(n-i) for i in range(1, n))

calc_T(4)

15.0

### Неукорененные деревья

Просто выкинем корневую вершину, а ее левого и правого ребенка соединим ребром. Тогда число таких деревьев размера n будет T(n-1)